예측 속도를 높이고, 트랜스포머 모델의 메모리 사용량 줄이는 네 가지 기술  
: 지식 정제 / 양자화 / 가지치기 / 그래프 최적화(ONNX 포맷과 ONNX 런타임 사용)

### 의도 탐지 예제

In [2]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [3]:
from transformers import pipeline

bert_ckpt = "transformersbook/bert-base-uncased-finetuned-clinc"
pipe = pipeline("text-classification", model=bert_ckpt)

C:\Users\knuyh\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
C:\Users\knuyh\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/8.18k [00:00<?, ?B/s]

C:\Users\knuyh\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\knuyh\.cache\huggingface\hub\models--transformersbook--bert-base-uncased-finetuned-clinc. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

C:\Users\knuyh\anaconda3\Lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [4]:
query = """Hey, I'd like to rent a vehicle from Nov 1st to Nov 15th in
Paris and I need a 15 passenger van"""
pipe(query) # 예측한 의도와 신뢰도 점수

[{'label': 'car_rental', 'score': 0.5490036010742188}]

### 벤치마크 클래스 만들기

In [5]:
from datasets import load_dataset

clinc = load_dataset("clinc_oos", "plus") # plus : 범위 밖의 훈련 샘플이 담긴 서브셋

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/15250 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3100 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5500 [00:00<?, ? examples/s]

In [6]:
sample = clinc["test"][42]
sample # 의도는 ID로 제공

{'text': 'transfer $100 from my checking to saving account', 'intent': 133}

In [7]:
intents = clinc["test"].features["intent"]
intents.int2str(sample["intent"]) # 의도 : ID를 문자열로 쉽게 매핑

'transfer'

In [8]:
import evaluate

accuracy_score = evaluate.load("accuracy")

In [14]:
import torch
import numpy as np
from pathlib import Path
from time import perf_counter


class PerformanceBenchmark:
    def __init__(self, pipeline, dataset, optim_type="BERT baseline"):
        self.pipeline = pipeline
        self.dataset = dataset
        self.optim_type = optim_type

    def compute_accuracy(self):
        preds, labels = [], []
        for example in self.dataset :
            pred = self.pipeline(example['text'])[0]['label']
            label = example['intent']
            preds.append(intents.str2int(pred))
            labels.append(label)
        accuracy = accuracy_score.compute(predictions = preds, references = labels)
        print(f"테스트 세트 정확도 - {accuracy['accuracy']:.3f}")
        return accuracy

    
    def compute_size(self):
        state_dict = self.pipeline.model.state_dict()
        # state_dict() : 모델의 층과 학습가능한 파라미터(가중치와 편향)를 매핑하는 dict 반환
        # 키 : BERT 층 / 값 : BERT 텐서 
        tmp_path = Path('model.pt')
        torch.save(state_dict, tmp_path)
        # 메가바이트 단위로 크기 계산
        size_mb = Path(tmp_path).stat().st_size / (1024*1024)
        # 임시 파일 삭제
        tmp_path.unlink()
        print(f"모델 크기 (MB) - {size_mb:.2f}")
        return {"size_mb": size_mb}
        

    def time_pipeline(self, query="What is the pin number for my account?"):  # 쿼리마다 평균적인 레이턴시 잼
        latencies = []
        # 워밍업
        for _ in range(10) :
            _ = self.pipeline(query)
        # 실행 측정
        for _ in range(100) :
            start_time = perf_counter()
            _ = self.pipeline(query)
            latency = perf_counter() - start_time
            latencies.append(latency)
        # 통계 계산
        time_avg_ms = 1000 * np.mean(latencies)
        time_std_ms = 1000 * np.std(latencies)
        print(f"평균 레이턴시 (ms) - {time_avg_ms:.2f} +\- {time_std_ms:.2f}")
        return {"time_avg_ms": time_avg_ms, "time_std_ms": time_std_ms}


    def run_benchmark(self):
        metrics = {}
        metrics[self.optim_type] = self.compute_size()
        metrics[self.optim_type].update(self.time_pipeline())
        metrics[self.optim_type].update(self.compute_accuracy())
        return metrics

In [15]:
# 기준 모델
pb = PerformanceBenchmark(pipe, clinc["test"])
perf_metrics = pb.run_benchmark()

모델 크기 (MB) - 418.15
평균 레이턴시 (ms) - 47.53 +\- 5.07
테스트 세트 정확도 - 0.867


### 지식 정제로 모델 크기 줄이기
* 지식 정제 트레이너 만들기

In [16]:
from transformers import TrainingArguments

class DistillationTrainingArguments(TrainingArguments):
    def __init__(self, *args, alpha=0.5, temperature=2.0, **kwargs):
        super().__init__(*args, **kwargs)
        self.alpha = alpha  # 정제 손실의 상대적인 가중치 제어
        self.temperature = temperature # 레이블의 확률 분포를 얼마나 완만하게 만들지 조절

In [17]:
import torch.nn as nn
import torch.nn.functional as F
from transformers import Trainer

In [18]:
class DistillationTrainer(Trainer):
    def __init__(self, *args, teacher_model=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.teacher_model = teacher_model

    def compute_loss(self, model, inputs, return_outputs=False):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        inputs = inputs.to(device)
        outputs_stu = model(**inputs)
        
        # 스튜던트의 크로스 엔트로피 손실과 로짓 추출
        loss_ce = outputs_stu.loss
        logits_stu = outputs_stu.logits
        
        # 티처의 로짓 추출
        with torch.no_grad():
            outputs_tea = self.teacher_model(**inputs)
            logits_tea = outputs_tea.logits
            
        # 확률을 부드럽게하고 정제 손실 계산
        loss_fct = nn.KLDivLoss(reduction="batchmean") # KL 발산
        loss_kd = self.args.temperature ** 2 * loss_fct(
            F.log_softmax(logits_stu / self.args.temperature, dim=-1),
            F.softmax(logits_tea / self.args.temperature, dim=-1))
        
        # 가중 평균된 스튜던트 손실 반환
        loss = self.args.alpha * loss_ce + (1. - self.args.alpha) * loss_kd
        return (loss, outputs_stu) if return_outputs else loss

* 좋은 스튜던트 초기화 선택하기
    * 레이턴시와 메모리 사용량을 줄이기 위해 스튜던트로 작은 모델 골라야
    * 경험 법칙에 의하면 티처와 스튜던트가 동일한 종류의 모델일 때 지식 정제 잘 동작함 -- BERT와 RoBERTa처럼 모델 종류가 다를 때 출력 임베딩 공간이 달라 스튜던트가 티처를 모방하는 데 방해가 됨

In [19]:
from transformers import AutoTokenizer

student_ckpt = "distilbert-base-uncased"
student_tokenizer = AutoTokenizer.from_pretrained(student_ckpt)

def tokenize_text(batch):
    return student_tokenizer(batch["text"], truncation=True)

clinc_enc = clinc.map(tokenize_text, batched=True, remove_columns=["text"])
clinc_enc = clinc_enc.rename_column("intent", "labels")

C:\Users\knuyh\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

C:\Users\knuyh\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\knuyh\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/15250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3100 [00:00<?, ? examples/s]

Map:   0%|          | 0/5500 [00:00<?, ? examples/s]

In [20]:
def compute_metrics(pred):
    predictions, labels = pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy_score.compute(predictions=predictions, references=labels)

In [46]:
batch_size = 48

finetuned_ckpt = "distilbert-base-uncased-finetuned-clinc"
student_training_args = DistillationTrainingArguments(
    output_dir='C:/Users/knuyh/Desktop', evaluation_strategy = "epoch",
    num_train_epochs=3, learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size, alpha=1, weight_decay=0.01,
    push_to_hub=True)
# 처음에는 alpha를 1로 지정해 티처로부터 어떤 신호도 받지 않고 성능 확인

In [47]:
student_training_args.logging_steps = len(clinc_enc['train']) // batch_size
student_training_args.disable_tqdm = False
student_training_args.save_steps = 1e9
student_training_args.log_level = 'error'

In [48]:
id2label = pipe.model.config.id2label
label2id = pipe.model.config.label2id

In [49]:
from transformers import AutoConfig

num_labels = intents.num_classes
student_config = (AutoConfig
                  .from_pretrained(student_ckpt, num_labels=num_labels,
                                   id2label=id2label, label2id=label2id))

In [50]:
from transformers import AutoModelForSequenceClassification

def student_init():
    return AutoModelForSequenceClassification.from_pretrained(student_ckpt, config=student_config)

In [51]:
teacher_ckpt = "transformersbook/bert-base-uncased-finetuned-clinc"
teacher_model = AutoModelForSequenceClassification.from_pretrained(teacher_ckpt, num_labels=num_labels)

In [ ]:
distilbert_trainer = DistillationTrainer(model_init=student_init,
    teacher_model=teacher_model, args=student_training_args,
    train_dataset=clinc_enc['train'], eval_dataset=clinc_enc['validation'],
    compute_metrics=compute_metrics, tokenizer=student_tokenizer)

distilbert_trainer.train()

Epoch,Training Loss,Validation Loss


In [ ]:
finetuned_ckpt = "transformersbook/distilbert-base-uncased-finetuned-clinc"
pipe = pipeline("text-classification", model=finetuned_ckpt)

In [ ]:
optim_type = "DistilBERT"
pb = PerformanceBenchmark(pipe, clinc["test"], optim_type=optim_type)
perf_metrics.update(pb.run_benchmark())

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def plot_metrics(perf_metrics, current_optim_type):
    df = pd.DataFrame.from_dict(perf_metrics, orient='index')

    for idx in df.index:
        df_opt = df.loc[idx]
        if idx == current_optim_type:
            plt.scatter(df_opt["time_avg_ms"], df_opt["accuracy"] * 100,
                        alpha=0.5, s=df_opt["size_mb"], label=idx,
                        marker='$\u25CC$')
        else:
            plt.scatter(df_opt["time_avg_ms"], df_opt["accuracy"] * 100,
                        s=df_opt["size_mb"], label=idx, alpha=0.5)

    legend = plt.legend(bbox_to_anchor=(1,1))
    for handle in legend.legend_handles:
        handle.set_sizes([20])

    plt.ylim(80,90)
    # 가장 느린 모델을 사용해 x 축 범위 지정
    xlim = int(perf_metrics["BERT baseline"]["time_avg_ms"] + 3)
    plt.xlim(1, xlim)
    plt.ylabel("Accuracy (%)")
    plt.xlabel("Average latency (ms)")
    plt.show()

plot_metrics(perf_metrics, optim_type)

작은 모델을 사용해 평균 레이턴시를 크게 줄였음  
티처의 정제 손실 추가해보기

* 옵투나로 좋은 하이퍼파라미터 찾기

In [ ]:
def objective(trial):
    x = trial.suggest_float("x", -2, 2) # 균등하게 샘플링할 파라미터 범위 지정
    y = trial.suggest_float("y", -2, 2)
    return (1 - x) ** 2 + 100 * (y - x ** 2) ** 2

In [ ]:
import optuna

study = optuna.create_study()
study.optimize(objective, n_trials=1000)

In [ ]:
study.best_params # 스터디 완료 후 최상의 파라미터 얻음

In [ ]:
def hp_space(trial):
    return {"num_train_epochs": trial.suggest_int("num_train_epochs", 5, 10),
        "alpha": trial.suggest_float("alpha", 0, 1),
        "temperature": trial.suggest_int("temperature", 2, 20)}

In [ ]:
best_run = distilbert_trainer.hyperparameter_search(
    n_trials=20, direction="maximize", hp_space=hp_space)
# 최대화된 목적 함수의 값과 하이퍼파라미터 담음

In [ ]:
print(best_run)

In [ ]:
for k,v in best_run.hyperparameters.items():
    setattr(student_training_args, k, v)

# 정제된 모델을 저장할 새로운 저장소를 정의합니다
distilled_ckpt = "distilbert-base-uncased-distilled-clinc"
student_training_args.output_dir = distilled_ckpt

# 최적의 매개변수로 새로운 Trainer 생성
distil_trainer = DistillationTrainer(model_init=student_init,
    teacher_model=teacher_model, args=student_training_args,
    train_dataset=clinc_enc['train'], eval_dataset=clinc_enc['validation'],
    compute_metrics=compute_metrics, tokenizer=student_tokenizer)

distil_trainer.train();

스튜던트의 매개변수 개수는 거의 티처의 절반이지만  
티처의 정확도에 버금가는 스튜던트 훈련

* 정제 모델 벤치마크 수행하기

In [ ]:
pipe = pipeline("text-classification", model=distilled_ckpt)
optim_type = "Distillation"
pb = PerformanceBenchmark(pipe, clinc["test"], optim_type=optim_type)
perf_metrics.update(pb.run_benchmark())

In [ ]:
plot_metrics(perf_metrics, optim_type)

모델 크기와 레이턴시는 기본적으로 DistilBERT에 비해 달라지지 않았지만, 정확도는 향상되고 티처의 성능도 뛰어난다.

### 양자화로 모델 속도 높이기
지식 정제를 사용해 티처의 정보를 작은 스튜던트 모델로 전송해 추론 실행 시, 계산 비용과 메모리 사용량 줄이는 방법  

<br>
양자화 방식 : 계산량을 줄이는 대신 가중치와 활성화 출력을 32비트 부동 소수점(FP32)이 아닌 8비트 정수(INT8) 같이 정밀도가 낮은 데이터 타입으로 변환해 계산 효율적으로 수행

* 아핀 변환 : 고정 소수점 숫자를 역양자화해 부동 소수점으로 되돌려야 하는 매핑

In [ ]:
state_dict = pipe.model.state_dict()
weights = state_dict["distilbert.transformer.layer.0.attention.out_lin.weight"]
plt.hist(weights.flatten().numpy(), bins=250, range=(-0.3,0.3), edgecolor="C0")
plt.show()

In [ ]:
zero_point = 0
scale = (weights.max() - weights.min()) / (127 - (-128))

In [ ]:
(weights / scale + zero_point).clamp(-128, 127).round().char()

In [ ]:
from torch import quantize_per_tensor

dtype = torch.qint8
quantized_weights = quantize_per_tensor(weights, scale, zero_point, dtype)
quantized_weights.int_repr()

In [ ]:
# 트랜스포머 가중치에서 양자화 효과
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes,mark_inset

# 히스토그램 그리기
fig, ax = plt.subplots()
ax.hist(quantized_weights.dequantize().flatten().numpy(),
         bins=250, range=(-0.3,0.3), edgecolor="C0");
# 확대 그림 만들기
axins = zoomed_inset_axes(ax, 5, loc='upper right')
axins.hist(quantized_weights.dequantize().flatten().numpy(),
         bins=250, range=(-0.3,0.3));
x1, x2, y1, y2 = 0.05, 0.1, 500, 2500
axins.set_xlim(x1, x2)
axins.set_ylim(y1, y2)
axins.axes.xaxis.set_visible(False)
axins.axes.yaxis.set_visible(False)
mark_inset(ax, axins, loc1=2, loc2=4, fc="none", ec="0.5")
plt.show()

In [ ]:
%%timeit
weights @ weights

In [ ]:
from torch.nn.quantized import QFunctional

q_fn = QFunctional()

In [ ]:
%%timeit
q_fn.mul(quantized_weights, quantized_weights)

In [ ]:
# 가중치 텐서와 양자화된 텐서 저장크기 비교
import sys

sys.getsizeof(weights.untyped_storage()) / sys.getsizeof(quantized_weights.untyped_storage())

보통 선형 층만 양자화한다.  
양자화에서 주의할 점 - 모델에 있는 모든 연산에서 정밀도를 바꾸면 모델의 계산 그래프 각 지점에서 작은 변동 생김
<br>

[심층 신경망에서 사용하는 양자화 방법]
- 동적 양자화
    * 훈련 도중에 아무것도 바뀌지 않고 추론 과정에만 적응
    * 모델 가중치가 추론 전에 INT8로 변환
    * 가중치 외에 모델의 활성화도 양자화되는데, 이 양자화가 즉석에서 일어남
    * 부동 소수점 포맷으로 활성화를 메모리에 쓰고 읽어 정수와 부동 소수점 간의 변환이 성능 병목이 되는 경우도 있음
    
- 정적 양자화
    * 즉석에서 활성화를 양자화하지 않고 양자화 체계를 사전에 계산해 부동 소수점 변환을 피함
    * 추론에 앞서 대표 샘플 데이터에서 활성화 패턴을 관찰해 수행한 후 이상적인 양자화 체계를 계산해 저장
    * 훈련과 추론 과정에서 정밀도 차이로 모델 성능이 떨어짐
    
- 양자화를 고려한 훈련
    * 가짜로 FP32 값을 양자화해 훈련 중에 양자화의 효과 시뮬레이션
    * 훈련할 때 INT8 대신 FP32 반올림해 양자화 효과 흉내냄
    * 정방향 패스와 역방향 패스에서 모두 적용되며 정적 양자화와 동적 양자화 사용해 모델 성능 향상시킴

In [ ]:
# 동적 양자화
from torch.quantization import quantize_dynamic

model_ckpt = "transformersbook/distilbert-base-uncased-distilled-clinc"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = (AutoModelForSequenceClassification
         .from_pretrained(model_ckpt).to("cpu"))

model_quantized = quantize_dynamic(model, {nn.Linear}, dtype=torch.qint8)

모델 정확도에 미치는 영향 거의 없음 (양자화 특징)

### 양자화된 모델의 벤치마크 수행하기

In [ ]:
pipe = pipeline("text-classification", model=model_quantized,
                tokenizer=tokenizer)
optim_type = "Distillation + quantization"
pb = PerformanceBenchmark(pipe, clinc["test"], optim_type=optim_type)
perf_metrics.update(pb.run_benchmark())

In [ ]:
plot_metrics(perf_metrics, optim_type)

양자화된 모델의 크기가 정제된 모델의 거의 절반이고 심지어 성능도 약간 향상됐다.

### ONNX와 ONNX 런타임으로 추론 최적화하기
* ONNX는 변경 불가능한 연산 규격을 그룹화하기 위해 연산자 집합 사용

In [ ]:
import os
from psutil import cpu_count

os.environ["OMP_NUM_THREADS"] = f"{cpu_count()}"
os.environ["OMP_WAIT_POLICY"] = "ACTIVE" # 대기 스레드 활성 상태로 지정(CPU프로세서 사이클 사용)

In [ ]:
from transformers.convert_graph_to_onnx import convert

model_ckpt = "transformersbook/distilbert-base-uncased-distilled-clinc"
onnx_model_path = Path("onnx/model.onnx")
convert(framework="pt", model=model_ckpt, tokenizer=tokenizer,
        output=onnx_model_path, opset=12, pipeline_name="text-classification")

In [ ]:
from onnxruntime import (GraphOptimizationLevel, InferenceSession,
                         SessionOptions)

def create_model_for_provider(model_path, provider="CPUExecutionProvider"):
    options = SessionOptions()
    options.intra_op_num_threads = 1
    options.graph_optimization_level = GraphOptimizationLevel.ORT_ENABLE_ALL
    session = InferenceSession(str(model_path), options, providers=[provider])
    session.disable_fallback()
    return session

In [ ]:
onnx_model = create_model_for_provider(onnx_model_path)

In [ ]:
inputs = clinc_enc["test"][:1]
del inputs["labels"]
logits_onnx = onnx_model.run(None, inputs)[0]
logits_onnx.shape

In [ ]:
np.argmax(logits_onnx) # 예측

In [ ]:
clinc_enc["test"][0]["labels"] # 정답

In [ ]:
from scipy.special import softmax

class OnnxPipeline:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer

    def __call__(self, query):
        model_inputs = self.tokenizer(query, return_tensors="pt")
        inputs_onnx = {k: v.cpu().detach().numpy()
                       for k, v in model_inputs.items()}
        logits = self.model.run(None, inputs_onnx)[0][0, :]
        probs = softmax(logits)
        pred_idx = np.argmax(probs).item()
        return [{"label": intents.int2str(pred_idx), "score": probs[pred_idx]}]

In [ ]:
pipe = OnnxPipeline(onnx_model, tokenizer)
pipe(query)

In [ ]:
class OnnxPerformanceBenchmark(PerformanceBenchmark):
    def __init__(self, *args, model_path, **kwargs):
        super().__init__(*args, **kwargs)
        self.model_path = model_path

    def compute_size(self):
        size_mb = Path(self.model_path).stat().st_size / (1024 * 1024)
        print(f"모델 크기 (MB) - {size_mb:.2f}")
        return {"size_mb": size_mb}

In [ ]:
optim_type = "Distillation + ORT"
pb = OnnxPerformanceBenchmark(pipe, clinc["test"], optim_type,
                              model_path="onnx/model.onnx")
perf_metrics.update(pb.run_benchmark())

In [ ]:
plot_metrics(perf_metrics, optim_type)

레이턴시 향상됨

ORT모델을 양자화하기 위해 세 가지 방법 제공  
이 중 정제 모델에 동적 양자화 적용

In [ ]:
from onnxruntime.quantization import quantize_dynamic, QuantType

model_input = "onnx/model.onnx"
model_output = "onnx/model.quant.onnx"
quantize_dynamic(model_input, model_output, weight_type=QuantType.QInt8)

In [ ]:
onnx_quantized_model = create_model_for_provider(model_output)
pipe = OnnxPipeline(onnx_quantized_model, tokenizer)
optim_type = "Distillation + ORT (quantized)"
pb = OnnxPerformanceBenchmark(pipe, clinc["test"], optim_type,
                              model_path=model_output)
perf_metrics.update(pb.run_benchmark())

In [ ]:
plot_metrics(perf_metrics, optim_type)

ORT 양자화는 일반 양자화로 얻은 모델(Distillation+quantization)에 비해 모델 크기와 레이턴시 30% 가량 줄임  
ONNX는 임베딩 층도 양자화하기 때문

### 가중치 가지치기로 희소한 모델 만들기
양자화 같은 방법은 표현 정밀도를 낮춰 모델 크기를 줄인다.  
* 가중치 가지치기 : 일부 가중치를 제거해 크기를 줄이는 전략
    * 절댓값 가지치기 : 가중치 절댓값 크기에 따라 점수 계산
        * 계산량이 많다.
        * 두 개의 가중치 클러스터 만듦
        * 각 가중치의 중요도가 현재 작업과 직접적으로 관련된 순수한 지도 학습을 위해 고안됐다는 문제
        * 미세 튜닝 작업에서 중요한 가중치 삭제될 가능성
    * 이동 가지치기 : 미세 튜닝하는 동안 점진적으로 가중치 제거해 모델을 점차 희소하게 만듦
        * 미세 튜닝동안 가중치와 점수 모두 학습됨
        * 완만한 분포 만듦

In [ ]:
# 절댓값 가지치기에 사용되는 세제곱 희소성 스케줄러
def _sparsity(t, t_0=0, dt=1, s_i=0, s_f=0.9, N=100):
    return s_f + (s_i - s_f) * (1 - (t - t_0) / (N * dt))**3

steps = np.linspace(0,100,100)
values = [_sparsity(t) for t in steps]

fig, ax = plt.subplots()
ax.plot(steps, values)
ax.set_ylim(0,1)
ax.set_xlim(0,100)
ax.set_xlabel("Pruning step")
ax.set_ylabel("Sparsity")
plt.grid(linestyle="dashed")
plt.show()